In [1]:
import pandas as pd
import numpy as np 
from numpy.random import randint, random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from collections import Counter


In [2]:
best_seed = {2013: 74, 2014: 58, 2015: 94, 2016: 18, 2017: 66, 2018: 1, 2019: 42, 2020: 28}

In [3]:
preditores = {}

In [4]:
for ano in range(2013,2021):
    seed = best_seed[ano]
    df = pd.read_csv(f"datasets/{ano}.csv")
    X, y = df.iloc[:,2:-1], df.iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = seed) 
    
    
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)



    classificador = LinearSVC(dual=False, max_iter=3000)
    ova = OneVsRestClassifier(classificador)

    ova.fit(X_train, y_train)

    preditores[ano] = ova
    
    acc = accuracy_score(y_test, ova.predict(X_test))
    print(f"Ano {ano}\t acurácia {acc}")
    

Ano 2013	 acurácia 0.6052631578947368
Ano 2014	 acurácia 0.618421052631579
Ano 2015	 acurácia 0.631578947368421
Ano 2016	 acurácia 0.6578947368421053
Ano 2017	 acurácia 0.5921052631578947
Ano 2018	 acurácia 0.6842105263157895
Ano 2019	 acurácia 0.6447368421052632
Ano 2020	 acurácia 0.5757575757575758


In [5]:
previsoes = pd.read_csv("datasets/previsoes.csv")

# Criando a tabela com os dados que temos agora

In [6]:
tabela = []

for time in df.Mandante.unique():
    idx_mandante = df.query(f"Mandante == '{time}'").index.max()
    idx_visitante = df.query(f"Visitante == '{time}'").index.max()
    if idx_mandante > idx_visitante:
        pontos, saldo = df.loc[idx_mandante, "pontos_mandante":"saldo_mandante"].values
    else:
        pontos, saldo = df.loc[idx_visitante, "pontos_visitante":"saldo_visitante"].values

    tabela.append([time, pontos, saldo])



In [7]:
df_tabela = pd.DataFrame(tabela, columns=["time", "pontos", "saldo"]).set_index("time")

In [8]:
def get_diff_gols(classe):
    """Função que gera diferença de gols baseada em distribuições aleatórias empíricas"""
    if classe == 0:
        return 0
    else:
        x = random()
        kerneis = {-1: np.array([  0.61797753, 0.88764045, 0.96629213, 1]), 
                        1: np.array([0.55319149, 0.82978723, 0.95035461, 0.9929078 , 1 ])}
        distribuicao = kerneis[classe]
        saldo = sum(distribuicao < x) + 1
        return classe*saldo
                

In [9]:
def get_pontos_saldo(time):
    return df_tabela.loc[time].values

def update_tabela(mandante, visitante, classe):
    """Conforme o resultado da predição, atualiza os valores na tabela"""
    diff_gols = get_diff_gols(classe) 
    pontuações = {-1: (0,3), 0: (1,1), 1: (3,0)}
    pt_mand, pt_visi = pontuações[classe]
    
    df_tabela.loc[mandante] += np.array([pt_mand,diff_gols])
    df_tabela.loc[visitante] += np.array([pt_visi,-diff_gols])

In [10]:
def complete(row):
    """Retorna o array já escalado, pronto para ser inserido no preditor"""
    mandante = row[0]
    visitante = row[1] 
    saida = np.array([*row[2:4], *get_pontos_saldo(mandante), row[6], *get_pontos_saldo(visitante)])
    return scaler.transform([saida])

# Loop das previsões

In [11]:
for x in previsoes.values:
    mandante = x[0]
    visitante = x[1] 
    array_previsao = complete(x)
    resultados = [ y.predict(array_previsao)[0] for y in preditores.values() ]
    contador = Counter(resultados)
    resultado = contador.most_common(1)[0]
    classe = resultado[0] 
    update_tabela(mandante, visitante, classe)

In [12]:
df_tabela.sort_values(by="pontos", ascending=False)

,pontos,saldo
time,,
Atlético-mg,79,24
Internacional,71,24
São paulo,70,23
Flamengo,67,17
Grêmio,63,14
Palmeiras,61,16
Fluminense,60,8
Bragantino,60,14
Ceará,54,3
